In [1]:
import numpy as np
import pandas as pd
import warnings
import time
warnings.filterwarnings("ignore")

In [2]:
# Read data and labels
data = pd.read_csv("../data/kmeans_data/data.csv", header = None)
labels = pd.read_csv('../data/kmeans_data/label.csv', header = None)

data.shape

(10000, 784)

In [12]:
# Define class to implement kmeans
class KMeans():
    def __init__(self, k, distance, max_iter, evaluation_criteria, majority_voting = False):
        # Store in the class
        self.k = k
        self.max_iter = max_iter
        self.distance = distance
        self.evaluation_criteria = evaluation_criteria
        self.majority_voting = majority_voting

        # Decide which distance to use
        if distance == "euclidean":
            self.distance_function = self.__euclidean
        elif distance == 'cosine':
            self.distance_function = self.__cosine
        else:
            self.distance_function = self.__jaccard

    # Compute euclidean function
    def __euclidean(self, x, centroids):
        euclid = []
        for center in centroids:
            euclid.append(np.sqrt(np.sum((x - center)**2)))
        return euclid

    # Compute cosine function
    def __cosine(self, x, centroids):
        cosine = []
        for center in centroids:
            cosine.append(np.dot(x.T, center) / (np.linalg.norm(x) * np.linalg.norm(center)))
        return cosine

    # Compute jaccard function
    def __jaccard(self, x, centroids):
        jaccard = []

        for center in centroids:
            numerator = np.sum(np.minimum(x, center))
            denominator = np.sum(np.maximum(x, center))
            temp_jaccard = 1 - (numerator / denominator)
            jaccard.append(temp_jaccard)
        
        return jaccard
    
    # Compute Sum Of Squared Error
    def __sumOfSquares(self, x, center):
        sse = 0
        for i in range(len(center)):
            sse += (center[i] - x[i])**2
        return sse

    # Compute Accuracy
    def __accuracy(self, X_train, labels):
        correct = 0
        if not self.majority_voting:
            for index, row in enumerate(X_train.iterrows()):
                flag = True
                for cluster_row in self.clusters[labels.iloc[index].values[0]]:
                    if np.not_equal(cluster_row, row[1]).any():
                        flag = False
                        break
                    if flag:
                        correct += 1
        else:
            for index, row in enumerate(X_train.iterrows()):
                cluster_label = labels.iloc[index].values[0]
                cluster_rows = self.clusters[cluster_label]

                # Use majority voting to determine the predicted label
                predicted_label = max(set(cluster_rows), key=cluster_rows.count)

                # Check if the predicted label matches the actual label
                if np.array_equal(predicted_label, row[1]):
                    correct += 1
        return 100 * (correct / X_train.shape[0])

    # Fit the model and find the centroids
    def fit(self, X_train, labels):
        # Initially, randomly select k datapoints as centrod.
        self.centroids = []

        # For jaccard, we need entire row instead of just one element
        for _ in range(self.k):
            self.centroids.append(np.array(X_train.sample()))
        
        # Reshape
        self.centroids = np.array(self.centroids).reshape(self.k, -1)
        # print("Centroids array:", np.array(centroids).shape)

        # Initialize few required variables
        iterations = 0
        prev_centroids = None
        prev_sse = float('inf')
        current_sse = 0

        # Iteratively update the centroids
        # Start the timer
        start = time.time()
        while True:
            print("Iteration Number: ", iterations + 1)

            # Iterate over all rows
            self.clusters = [[] for _ in range(self.k)]
            for row in X_train.iterrows():
                distance = self.distance_function(np.array(row[1]), np.array(self.centroids))
                cluster_number = np.argmin(distance)
                self.clusters[cluster_number].append(row[1])
            
            # Update the new centroid
            self.centroids = []
            for cluster in self.clusters:
                cluster = np.array(cluster)
                if len(cluster) == 0:
                    self.centroids.append(np.zeros(784))
                else:
                    self.centroids.append(np.mean(cluster, axis = 0))

            # Compute current sse
            current_sse = self.evaluate(X_train, labels) / 10000

            if prev_sse <= current_sse:
                print("Exiting due to increament in sse")
                print("Number of Iterations:", iterations + 1)
                print("SSE:", current_sse / 10000)
                end = time.time()
                time_taken = end - start
                print('Time Taken (in seconds):', time_taken)
                break
            
            # if prev_centroids != None:
            #     if np.equal(self.centroids, prev_centroids).any():
            #         print("Exiting due to constant centroid")
            #         print('Number of Iterations:', iterations + 1)
            #         end = time.time()
            #         time_taken = end - start
            #         print('Time Taken (in seconds):', time_taken)
            #         print("SSE:", current_sse / 10000)
            #         break
            
            if iterations >= self.max_iter:
                print("Exiting due to maximum iterations")
                print("Number of iterations:", iterations + 1)
                end = time.time()
                time_taken = end - start
                print('Time Taken (in seconds):', time_taken)
                print("SSE:", current_sse / 10000)
                break

            # Save the previous centroids so that current ones can be overwritten
            prev_centroids = self.centroids
            prev_sse = current_sse

            # Increment the number of iterations
            iterations += 1

    # Evaluate the generated centroids
    def evaluate(self, X_train, labels):
        if self.evaluation_criteria == 'sse':
            sse = 0
            for index, cluster in enumerate(self.clusters):
                for row in cluster:
                    sse += self.__sumOfSquares(row, self.centroids[index])
            return sse
        else:
            return self.__accuracy(X_train, labels)

In [8]:
# Cosine Similarity, with evaluation_criteria = 'sse'
kmeans_cosine = KMeans(k = 10, distance = 'cosine', max_iter = 10, evaluation_criteria = 'sse')

# Fit model
kmeans_cosine.fit(data, labels)

# Compute sse
sse_cosine = kmeans_cosine.evaluate(data, labels)

# Display results
print("Normalized Sum of Squares for Cosine Distance:", sse_cosine / data.shape[0])

Iteration Number:  1
Iteration Number:  2
Exiting due to increament in sse
Number of Iterations: 2
SSE: 3436457.369859137
Time Taken (in seconds): 37.748679637908936
Normalized Sum of Squares for Euclidean Distance: 3436457.369859137


In [9]:
# Jaccard distance, with evaluation_criteria as "sse"
kmeans_jaccard = KMeans(k = 10, distance = 'jaccard', max_iter = 10, evaluation_criteria = 'sse')

# Fit model
kmeans_jaccard.fit(data, labels)

# Compute sse
sse_jaccard = kmeans_jaccard.evaluate(data, labels)

# Display results
print("Normalized Sum of Squares for Jaccard Distance:", sse_jaccard / data.shape[0])

Iteration Number:  1
Iteration Number:  2
Iteration Number:  3
Iteration Number:  4
Iteration Number:  5
Iteration Number:  6
Iteration Number:  7
Iteration Number:  8
Iteration Number:  9
Iteration Number:  10
Iteration Number:  11
Exiting due to maximum iterations
Number of iterations: 11
Time Taken (in seconds): 235.20397305488586
SSE: 2585156.0965118036
Normalized Sum of Squares for Jaccard Distance: 2585156.0965118036


In [10]:
# Euclidean distance, with evaluation_criteria as "sse"
kmeans_euclid = KMeans(k = 10, distance = 'euclidean', max_iter = 10, evaluation_criteria = 'sse')

# Fit model
kmeans_euclid.fit(data, labels)

# Compute sse
sse_euclid = kmeans_euclid.evaluate(data, labels)

# Display results
print("Normalized Sum of Squares for Euclidean Distance:", sse_euclid / data.shape[0])

Iteration Number:  1
Iteration Number:  2
Iteration Number:  3
Iteration Number:  4
Iteration Number:  5
Iteration Number:  6
Iteration Number:  7
Iteration Number:  8
Iteration Number:  9
Iteration Number:  10
Iteration Number:  11
Exiting due to maximum iterations
Number of iterations: 11
Time Taken (in seconds): 223.31162881851196
SSE: 2546327.4311793186
Normalized Sum of Squares for Euclidean Distance: 2546327.4311793186


In [ ]:
# Euclidean Distance, with evaluation_criteria = 'accuracy'
kmeans_euclid_acc = KMeans(k = 10, distance = 'euclidean', max_iter = 10, evaluation_criteria = 'accuracy')

# Fit Model
kmeans_euclid_acc.fit(data)

# Compute accuracy
accuracy = kmeans_euclid_acc.evaluate(data, labels)

# Display results
print('Accuracy for Euclidean Distance:', accuracy)

In [ ]:
# Cosine Distance, with evaluation_criteria = 'accuracy'
kmeans_cosine_acc = KMeans(k = 10, distance = 'cosine', max_iter = 10, evaluation_criteria = 'accuracy')

# Fit Model
kmeans_cosine_acc.fit(data)

# Compute accuracy
accuracy = kmeans_cosine_acc.evaluate(data, labels)

# Display results
print('Accuracy for Cosine Similarity:', accuracy)

In [ ]:
# Jaccard Distance, with evaluation_criteria = 'accuracy'
kmeans_jaccard_acc = KMeans(k = 10, distance = 'jaccard', max_iter = 10, evaluation_criteria = 'accuracy')

# Fit Model
kmeans_jaccard_acc.fit(data)

# Compute accuracy
accuracy = kmeans_jaccard_acc.evaluate(data, labels)

# Display results
print('Accuracy for Jaccard Similarity:', accuracy)

In [15]:
kmeans_cosine_acc_m = KMeans(k = 10, distance = 'cosine', max_iter = 10, evaluation_criteria = 'accuracy', majority_voting = True)

# Fit Model
kmeans_cosine_acc_m.fit(data, labels)

# Compute accuracy
accuracy = kmeans_cosine_acc_m.evaluate(data, labels)

# Display results
print('Accuracy for Jaccard Similarity:', accuracy)

Iteration Number:  1


TypeError: unhashable type: 'Series'